In [1]:
import pandas as pd
import xarray as xr
from erddapy import ERDDAP
import datetime
import time

import sys
sys.path.append('/home/jovyan/repos/hemasphere/botpt/pangeo-forge-recipes')
import pangeo_forge_recipes


from pangeo_forge_recipes.patterns import ConcatDim, FilePattern, MergeDim
from pangeo_forge_recipes.recipes import XarrayZarrRecipe

#pip install fsspec-reference-maker

In [2]:
server = 'http://erddap.dataexplorer.oceanobservatories.org/erddap'
protocol = 'tabledap'

e = ERDDAP(
    server=server,
    protocol=protocol
)

In [3]:
dataset_id = e.dataset_id = 'ooi-rs03ccal-mj03f-05-botpta301'

In [4]:
variables = e.variables = ['time', 'latitude', 'longitude', 'botpres', 'botsflu_meandepth', 'botsflu_meanpres']

In [5]:
years = e.constraints = {
    'time>=': '2014-08-29T00:00:00Z',
    'time<=': '2014-09-02T00:00:00Z',
}

In [6]:
startYear = datetime.datetime(2014, 8, 29, 0, 0, 0)
endYear = datetime.datetime(2014, 9, 2, 0, 0, 0)
unixStart = time.mktime(startYear.timetuple())
unixEnd = time.mktime(endYear.timetuple())

In [7]:
print(unixStart)
print(unixEnd)

1409270400.0
1409616000.0


In [8]:
url = e.get_download_url()

In [9]:
url

'http://erddap.dataexplorer.oceanobservatories.org/erddap/tabledap/ooi-rs03ccal-mj03f-05-botpta301.html?time,latitude,longitude,botpres,botsflu_meandepth,botsflu_meanpres&time>=1409270400.0&time<=1409616000.0'

In [10]:
!{'wget '+url}

/bin/bash: =1409616000.0: No such file or directory

real	0m0.012s
user	0m0.000s
sys	0m0.000s


In [11]:
time_concat_dim = ConcatDim("time", years, nitems_per_file=1)
time_concat_dim

ConcatDim(name='time', nitems_per_file=1)

In [12]:
url_format = "http://erddap.dataexplorer.oceanobservatories.org/erddap/tabledap/{dataset_id}.nc?{variables}&time%3E={unixStart}&time%3C={unixEnd}"

In [13]:
def make_url(dataset_id, variables, unixStart, unixEnd):
    return url_format.format(dataset_id = dataset_id, variables = variables, unixStart = unixStart,  unixEnd=unixEnd)

In [14]:
make_url(dataset_id, variables, unixStart, unixEnd)

"http://erddap.dataexplorer.oceanobservatories.org/erddap/tabledap/ooi-rs03ccal-mj03f-05-botpta301.nc?['time', 'latitude', 'longitude', 'botpres', 'botsflu_meandepth', 'botsflu_meanpres']&time%3E=1409270400.0&time%3C=1409616000.0"

In [15]:
url

'http://erddap.dataexplorer.oceanobservatories.org/erddap/tabledap/ooi-rs03ccal-mj03f-05-botpta301.html?time,latitude,longitude,botpres,botsflu_meandepth,botsflu_meanpres&time>=1409270400.0&time<=1409616000.0'

In [16]:
pattern = FilePattern(make_url, time_concat_dim)
pattern

<FilePattern {'time': 2}>

In [17]:
for index, url in pattern.items():
    print(index)
    print(url)
    if unixStart in url and unixEnd in url:
        break

KeyError: 0

In [127]:
test = xr.open_dataset("test.nc")
test2 = xr.open_dataset("test2.nc")
# wget -O 

In [128]:
test

<xarray.Dataset>
Dimensions:            (row: 1437)
Dimensions without coordinates: row
Data variables:
    time               (row) datetime64[ns] ...
    latitude           (row) float64 ...
    longitude          (row) float64 ...
    botpres            (row) float64 ...
    botsflu_meandepth  (row) float64 ...
    botsflu_meanpres   (row) float64 ...
Attributes:
    cdm_data_type:                 TimeSeries
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             feedback@axiomdatascience.com
    contributor_name:              Axiom Data Science
    contributor_role:              processor
    contributor_role_vocabulary:   NERC
    contributor_url:               https://www.axiomdatascience.com
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3
    creator_institution:           Ocean Observatories Initiative (OOI)
    creator_name:                  Ocean Observatories Initiative (OOI)
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   https://oceanobservatories.org/
    defaultDataQuery:              bottilt_tdir,bottilt_ytlt,bottilt_xtlt,bot...
    Easternmost_Easting:           -130.00876
    featureType:                   TimeSeries
    geospatial_lat_max:            45.95484
    geospatial_lat_min:            45.95484
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -130.00876
    geospatial_lon_min:            -130.00876
    geospatial_lon_units:          degrees_east
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from Ocean Observatories Initia...
    id:                            104331
    infoUrl:                       https://sensors.ioos.us/#metadata/104331/s...
    institution:                   Ocean Observatories Initiative (OOI)
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         45.95484
    platform:                      buoy
    platform_name:                 Regional Cabled Array: Central Caldera: Me...
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    processing_level:              Level 2
    publisher_institution:         Ocean Observatories Initiative (OOI)
    publisher_name:                Ocean Observatories Initiative (OOI)
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 https://oceanobservatories.org/
    references:                    https://ooinet.oceanobservatories.org/data...
    sourceUrl:                     https://ooinet.oceanobservatories.org/data...
    Southernmost_Northing:         45.95484
    standard_name_vocabulary:      CF Standard Name Table v72
    station_id:                    104331
    summary:                       Timeseries data from 'Regional Cabled Arra...
    time_coverage_end:             2014-08-30T20:59:00Z
    time_coverage_start:           2014-08-29T20:59:00Z
    title:                         Regional Cabled Array: Central Caldera: Me...
    Westernmost_Easting:           -130.00876

In [129]:
test2

<xarray.Dataset>
Dimensions:            (row: 1440)
Dimensions without coordinates: row
Data variables:
    time               (row) datetime64[ns] ...
    latitude           (row) float64 ...
    longitude          (row) float64 ...
    botpres            (row) float64 ...
    botsflu_meandepth  (row) float64 ...
    botsflu_meanpres   (row) float64 ...
Attributes:
    cdm_data_type:                 TimeSeries
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             feedback@axiomdatascience.com
    contributor_name:              Axiom Data Science
    contributor_role:              processor
    contributor_role_vocabulary:   NERC
    contributor_url:               https://www.axiomdatascience.com
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3
    creator_institution:           Ocean Observatories Initiative (OOI)
    creator_name:                  Ocean Observatories Initiative (OOI)
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   https://oceanobservatories.org/
    defaultDataQuery:              bottilt_tdir,bottilt_ytlt,bottilt_xtlt,bot...
    Easternmost_Easting:           -130.00876
    featureType:                   TimeSeries
    geospatial_lat_max:            45.95484
    geospatial_lat_min:            45.95484
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -130.00876
    geospatial_lon_min:            -130.00876
    geospatial_lon_units:          degrees_east
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from Ocean Observatories Initia...
    id:                            104331
    infoUrl:                       https://sensors.ioos.us/#metadata/104331/s...
    institution:                   Ocean Observatories Initiative (OOI)
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         45.95484
    platform:                      buoy
    platform_name:                 Regional Cabled Array: Central Caldera: Me...
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    processing_level:              Level 2
    publisher_institution:         Ocean Observatories Initiative (OOI)
    publisher_name:                Ocean Observatories Initiative (OOI)
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 https://oceanobservatories.org/
    references:                    https://ooinet.oceanobservatories.org/data...
    sourceUrl:                     https://ooinet.oceanobservatories.org/data...
    Southernmost_Northing:         45.95484
    standard_name_vocabulary:      CF Standard Name Table v72
    station_id:                    104331
    summary:                       Timeseries data from 'Regional Cabled Arra...
    time_coverage_end:             2022-01-01T23:59:00Z
    time_coverage_start:           2022-01-01T00:00:00Z
    title:                         Regional Cabled Array: Central Caldera: Me...
    Westernmost_Easting:           -130.00876